In [13]:
import os
import pickle
import tsplib95
import csv
import time
import pandas as pd
import numpy as np
import networkx as nx
from networkx.algorithms.approximation.traveling_salesman import *

In [14]:
def get_tsp_graph(name) -> nx.Graph:
    # load from pkl file if exists
    graph_file_path = os.path.join("tsp_dataset", f"{name}.graph.pkl")
    if os.path.exists(graph_file_path):
        # print(f"Loading {name} from pkl {graph_file_path}")
        with open(graph_file_path, "rb") as file:
            G = pickle.load(file)
        return G.to_undirected()

    tsp_file_path = os.path.join("tsp_dataset", f"{name}.tsp")
    # print(f"Loading {name} from tsplib {tsp_file_path}")
    problem = tsplib95.load(tsp_file_path)
    G = problem.get_graph()

    # remove redundant edges
    if problem.edge_weight_type == "EXPLICIT" and problem.edge_weight_format == "FULL_MATRIX":
        for u in G.nodes:
            for v in G.nodes:
                if u > v:
                    G.remove_edge(u, v)

    # remove self loops
    loop_edges = list(nx.selfloop_edges(G))
    G.remove_edges_from(loop_edges)

    # save as pkl file
    with open(graph_file_path, "wb") as file:
        print(f"Saving {name} to pkl {graph_file_path}")
        pickle.dump(G, file)

    return G

In [15]:
tsp_dataset_file_path = os.path.join("tsp_dataset", "tsp_dataset.csv")
print(f"Loading tsp dataset from {tsp_dataset_file_path}")
tsp_dataset = pd.read_csv(tsp_dataset_file_path)

Loading tsp dataset from tsp_dataset\tsp_dataset.csv


In [16]:
def get_opt_tour_length(name):
    return tsp_dataset[tsp_dataset["name"] == name]["opt_tour_length"].values[0]

In [17]:
def get_tour_length(G: nx.graph, tour: list):
    tour_length = 0
    for i in range(len(tour) - 1):
        u, v = tour[i], tour[i + 1]
        w = G[u][v]["weight"]
        tour_length += w
    return tour_length

In [18]:
def get_apx_ratio(G: nx.graph, tour: list):
    return get_tour_length(G, tour) / get_opt_tour_length(G.name)

In [19]:
tsp_dataset

,name,dimension,opt_tour_length,type,comment,edge_weight_type,edge_weight_format
0,burma14,14,3323,TSP,14-Staedte in Burma (Zaw Win),GEO,FUNCTION
1,ulysses16,16,6859,TSP,Odyssey of Ulysses (Groetschel/Padberg),GEO,NaN
2,gr17,17,2085,TSP,17-city problem (Groetschel),EXPLICIT,LOWER_DIAG_ROW
3,gr21,21,2707,TSP,21-city problem (Groetschel),EXPLICIT,LOWER_DIAG_ROW
4,ulysses22,22,7013,TSP,Odyssey of Ulysses (Groetschel/Padberg),GEO,NaN
...,...,...,...,...,...,...,...
91,u2319,2319,234256,TSP,Drilling problem (Reinelt),EUC_2D,NaN
92,pr2392,2392,378032,TSP,2392-city problem (Padberg/Rinaldi),EUC_2D,NaN
93,pcb3038,3038,137694,TSP,Drilling problem (Junger/Reinelt),EUC_2D,NaN
94,fnl4461,4461,182566,TSP,Die 5 neuen Laender Deutschlands (Ex-DDR) (Bac...,EUC_2D,NaN


In [20]:
from mst import MST
st = MST()

In [21]:
def get_total_edge_weight(G: nx.graph):
    total_edge_weight = 0
    for u, v in G.edges:
        total_edge_weight += G[u][v]["weight"]
    return total_edge_weight

In [22]:
from simulated_annealing import SimulateAnnealing

In [23]:
with open("sa_results.csv", "a", newline="") as csvfile:
    writer = csv.writer(csvfile)

    for name in tsp_dataset["name"][58:]:
        print(name)

        G = get_tsp_graph(name)
        dimension = G.number_of_nodes()
        opt_tour_length = get_opt_tour_length(name)

        d = {
            "name": name,
            "dimension": dimension,
            "opt_tour_length": opt_tour_length,
        }

        best_tour = None
        best_tour_length = None
        best_tour_k = None

        best_sa_tour = None
        best_sa_tour_length = None
        best_sa_tour_k = None
        
        c_tour = None
        c_tour_length = None

        start_time = time.time()

        for k in range(1, 10):
            print(name, k) 
            tree = st.get_mst_k(G, k)
            tour = christofides(G, tree=tree)
            tour_length = get_tour_length(G, tour)
            
            if k == 1:
                c_tour = tour
                c_tour_length = tour_length

            if best_tour is None or tour_length < best_tour_length:
                best_tour = tour
                best_tour_length = tour_length
                best_tour_k = k

            sa = SimulateAnnealing(
                graph=G,
                initial_solution=tour,
                max_iterations=100000,
                initial_temperature=10000,
                cooling_rate=0.01,
            )
            sa_tour, sa_tour_cost = sa.run()
            sa_tour_length = get_tour_length(G, sa_tour)
            assert sa_tour_cost == sa_tour_length

            if best_sa_tour is None or sa_tour_length < best_sa_tour_length:
                best_sa_tour = sa_tour
                best_sa_tour_length = sa_tour_length
                best_sa_tour_k = k

        end_time = time.time()
        elapsed_time = end_time - start_time
        
        c_apx_ratio = c_tour_length / opt_tour_length
        best_apx_ratio = best_tour_length / opt_tour_length
        best_sa_apx_ratio = best_sa_tour_length / opt_tour_length

        d["c_tour_length"] = c_tour_length
        d["best_tour_length"] = best_tour_length
        d["best_sa_tour_length"] = best_sa_tour_length
        d["c_apx_ratio"] = c_apx_ratio
        d["best_apx_ratio"] = best_apx_ratio
        d["best_sa_apx_ratio"] = best_sa_apx_ratio
        d["best_tour_k"] = best_tour_k
        d["best_sa_tour_k"] = best_sa_tour_k
        d["elapsed_time"] = elapsed_time

        print(list(d.values()))
        writer.writerow(d.values())
        csvfile.flush()

        print()

rd400
rd400 1
rd400 2
rd400 3
rd400 4
rd400 5
rd400 6
rd400 7
rd400 8
rd400 9
['rd400', 400, 15281, 17513, 17257, 17257, 1.1460637392840782, 1.129310908971926, 1.129310908971926, 8, 8, 775.6145913600922]

fl417
fl417 1
fl417 2
fl417 3
fl417 4
fl417 5
fl417 6
fl417 7
fl417 8
fl417 9
['fl417', 417, 11861, 13610, 13045, 13045, 1.1474580558131693, 1.0998229491611162, 1.0998229491611162, 7, 7, 824.3679854869843]

gr431
gr431 1
gr431 2
gr431 3
gr431 4
gr431 5
gr431 6
gr431 7
gr431 8
gr431 9
['gr431', 431, 171414, 188815, 187553, 187553, 1.1015144620626087, 1.0941521696010827, 1.0941521696010827, 8, 8, 833.977212190628]

pr439
pr439 1
pr439 2
pr439 3
pr439 4
pr439 5
pr439 6
pr439 7
pr439 8
pr439 9
['pr439', 439, 107217, 119698, 118498, 118498, 1.11640877845864, 1.105216523499072, 1.105216523499072, 7, 7, 803.1478295326233]

pcb442
pcb442 1
pcb442 2
pcb442 3
pcb442 4
pcb442 5
pcb442 6
pcb442 7
pcb442 8
pcb442 9
['pcb442', 442, 50778, 55729, 55120, 55120, 1.0975028555673716, 1.0855094726062469,

KeyboardInterrupt: 